In [ ]:
%load_ext autoreload
%autoreload 2

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import matplotlib as mpl
import util
import warnings

import kinematics
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

In [ ]:
mpl.rc('font', size=14)

dosave = True
savedir = "./figures/"

In [ ]:
# FILE = "/exp/sbnd/data/users/munjung/osc/sbnd.df"
FILE = "/exp/sbnd/data/users/gputnam/gump_ep.df"


# DETECTOR = "SBND"
DETECTOR = "ICARUS"

# Utils

In [ ]:
def InFV(df):
    return util.InFV(df, 50, det=DETECTOR)

def InBeam(t):
    return (t > 0.) & (t < 1.800)

In [ ]:
def is_cosmic(df):
    return (df.slc.truth.pdg == -1)

def is_FV(df): 
    return (InFV(df.position))

def is_numu(df):
    return (np.abs(df.pdg) == 14)

def is_CC(df):
    return (df.iscc == 1)

def is_NC(df):
    return (df.iscc == 0)

def is_1p0pi(df):
     return (df.nmu == 1) & (df.np_50MeV == 1) & (df.npi == 0) & (df.npi0 == 0)

#     return (df.nmu_20MeV == 1) & (df.np_50MeV == 1) & (df.npi_40MeV == 0) & (df.npi0 == 0)

def is_signal(df):
    return is_numu(df) & is_CC(df) & is_1p0pi(df) & is_FV(df)

def is_outFV(df):
    return is_numu(df) & is_CC(df) & is_1p0pi(df) & np.invert(is_FV(df))

def is_othernumuCC(df):
    return is_numu(df) & is_CC(df) & np.invert(is_1p0pi(df)) & is_FV(df)

In [ ]:
def cut_all(cuts):
    ret = cuts[0]
    for c in cuts[1:]:
        ret = ret & c
    return ret

# Plotters

In [ ]:
mode_list = [0, 10, 1, 2, 3]
mode_labels = ['QE', 'MEC', 'RES', 'SIS/DIS', 'COH', "other"]
mode_colors = ["darkorchid", "royalblue", "forestgreen", "darkorange", "firebrick"]

def breakdown_mode(var, w, df, cuts):
    c = cut_all(cuts)

    ret = [var[(df.genie_mode == i) & c] for i in mode_list] 
    wret = [w[(df.genie_mode == i) & c] for i in mode_list] 
    return ret, wret


In [ ]:
top_labels = ["Signal",
              "Other $\\nu_\\mu$ CC",
              "$\\nu$ NC",
              "Out of FV",
              "Cosmic",
              "Other"]

top_colors = []
def breakdown_top(var, w, df, cuts):
    c = cut_all(cuts)
    ret = [var[is_signal(df) & c],
           var[is_othernumuCC(df) & c],
           var[is_NC(df) & c],
           var[is_outFV(df) & c],
           var[is_cosmic(df) & c],
           var[np.invert(is_signal(df) | is_othernumuCC(df) | is_NC(df) | is_outFV(df) | is_cosmic(df)) & c]
           ]
    wret = [w[is_signal(df) & c],
           w[is_othernumuCC(df) & c],
           w[is_NC(df) & c],
           w[is_outFV(df) & c],
           w[is_cosmic(df) & c],
           w[np.invert(is_signal(df) | is_othernumuCC(df) | is_NC(df) | is_outFV(df) | is_cosmic(df)) & c]
           ]
    return ret, wret

# Selection

In [ ]:
df = pd.read_hdf(FILE, "evt")
mcdf = pd.read_hdf(FILE, "mcnu")
hdr = pd.read_hdf(FILE, "hdr")

In [ ]:
if DETECTOR == "ICARUS":
    GOAL_POT = 3.41e20
    POTSTR = "$3.41\\times 10^{20}$ POT"
elif DETECTOR == "SBND":
    GOAL_POT = 6.6e20 / 3
    POTSTR = "$2.2\\times 10^{20}$ POT"

In [ ]:
POT = hdr.pot.sum()

df["weight"] = GOAL_POT / POT
mcdf["weight"] = GOAL_POT / POT
GOAL_POT / POT

## FV cut

In [ ]:
cuts = [InFV(df.slc.vertex)]

## Cosmic Rejection

In [ ]:
var = df.slc.nuid.crlongtrkdiry
bins = np.linspace(-1,1,21)

pvar, pw = breakdown_top(var, df.weight, df, cuts)
n, bins, _ = plt.hist(pvar, bins=bins, stacked=True, 
                      label=top_labels, weights=pw)
plt.ylabel("Candidates / %s" % POTSTR)
plt.title("%s Pandora Selection" % DETECTOR)
plt.tight_layout()
plt.legend()
plt.xlabel("CRLongTrkDirY")
plt.show();

if dosave:
    plt.savefig(savedir + "%s_fiducial_crlongtrkdiry.pdf" % DETECTOR)
    plt.savefig(savedir + "%s_fiducial_crlongtrkdiry.png" % DETECTOR)

In [ ]:
var = df.slc.nu_score
bins=np.linspace(0,1,21)

pvar, pw = breakdown_top(var, df.weight, df, cuts)
n, bins, _ = plt.hist(pvar, bins=bins, stacked=True, 
                      label=top_labels, weights=pw)
plt.xlabel("Pandora $\\nu$ Score")
plt.ylabel("Candidates / %s" % POTSTR)
plt.title("%s Pandora Selection" % DETECTOR)
plt.tight_layout()
plt.legend()
plt.show();

if dosave:
    plt.savefig(savedir + "%s_fiducial_nuscore.pdf" % DETECTOR)
    plt.savefig(savedir + "%s_fiducial_nuscore.png" % DETECTOR)


In [ ]:
# Traditional 
nu_score = (df.slc.nu_score > 0.5)
# f_match = (df.slc.fmatch.score < 7.0) & (InBeam(df.slc.fmatch.time))
cosmic_rejection = nu_score #& f_match

# CRUMBS
# crumbs = (df.slc_crumbs_result.score > 0)
# cosmic_rejection = (crumbs)

# df = df[cosmic_rejection]

In [ ]:
var = df.mu.pfp.trk.P.p_muon
bins=np.linspace(0,2,21)

pvar, pw = breakdown_top(var, df.weight, df, cuts)
n, bins, _ = plt.hist(pvar, bins=bins, stacked=True, 
                      label=top_labels, weights=pw)
plt.ylabel("Candidates / %s" % POTSTR)
plt.title("%s Pandora Selection" % DETECTOR)
plt.tight_layout()
plt.legend()
plt.xlabel("Muon Momentum [GeV/c]")
plt.show();
print("signal purity {:.2f} %".format(100*n[0].sum()/n[-1].sum()))


if dosave:
    plt.savefig(savedir + "%s_cosrej_muon_momentum.pdf" % DETECTOR)
    plt.savefig(savedir + "%s_cosrej_muon_momentum.png" % DETECTOR)



## Select 1mu1p topology

In [ ]:
# reject slices with any showers or 3rd tracks
twoprong_cut = (np.isnan(df.other_shw_length) & np.isnan(df.other_trk_length))

cuts += [twoprong_cut]

In [ ]:
var = df.mu.pfp.trk.P.p_muon
bins=np.linspace(0,2,21)

pvar, pw = breakdown_top(var, df.weight, df, cuts)
n, bins, _ = plt.hist(pvar, bins=bins, stacked=True, 
                      label=top_labels, weights=pw)

print("signal purity {:.2f} %".format(100*n[0].sum()/n[-1].sum()))

plt.ylabel("Candidates / %s" % POTSTR)
plt.title("%s Pandora Selection" % DETECTOR)
plt.tight_layout()
plt.legend()
plt.xlabel("Muon Momentum [GeV/c]")
plt.show();


if dosave:
    plt.savefig(savedir + "%s_otherobj_muon_momentum.pdf" % DETECTOR)
    plt.savefig(savedir + "%s_otherobj_muon_momentum.png" % DETECTOR)

In [ ]:
var = df.mu.pfp.trk.chi2pid.I2.chi2_muon
bins=np.linspace(0,100,21)

pvar, pw = breakdown_top(var, df.weight, df, cuts)
n, bins, _ = plt.hist(pvar, bins=bins, stacked=True, 
                      label=top_labels, weights=pw)

print("signal purity {:.2f} %".format(100*n[0].sum()/n[-1].sum()))

plt.ylabel("Candidates / %s" % POTSTR);
plt.title("%s Pandora Selection" % DETECTOR);
plt.tight_layout();
plt.legend();
plt.xlabel("Muon Candidate $\\chi^2_\\mu$");
plt.show();

if dosave:
    plt.savefig(savedir + "%s_otherobj_muon_chi2u.pdf" % DETECTOR);
    plt.savefig(savedir + "%s_otherobj_muon_chi2u.png" % DETECTOR);

In [ ]:
var = df.mu.pfp.trk.chi2pid.I2.chi2_proton
bins=np.linspace(0,300,21)

pvar, pw = breakdown_top(var, df.weight, df, cuts)
n, bins, _ = plt.hist(pvar, bins=bins, stacked=True, 
                      label=top_labels, weights=pw)

print("signal purity {:.2f} %".format(100*n[0].sum()/n[-1].sum()))

plt.ylabel("Candidates / %s" % POTSTR);
plt.title("%s Pandora Selection" % DETECTOR);
plt.tight_layout();
plt.legend();
plt.xlabel("Muon Candidate $\\chi^2_p$");
plt.show();

if dosave:
    plt.savefig(savedir + "%s_otherobj_muon_chi2p.pdf" % DETECTOR);
    plt.savefig(savedir + "%s_otherobj_muon_chi2p.png" % DETECTOR);

In [ ]:
var = df.p.pfp.trk.chi2pid.I2.chi2_muon
bins=np.linspace(0,100,21)

pvar, pw = breakdown_top(var, df.weight, df, cuts)
n, bins, _ = plt.hist(pvar, bins=bins, stacked=True, 
                      label=top_labels, weights=pw)

print("signal purity {:.2f} %".format(100*n[0].sum()/n[-1].sum()))

plt.ylabel("Candidates / %s" % POTSTR);
plt.title("%s Pandora Selection" % DETECTOR);
plt.tight_layout();
plt.legend();
plt.xlabel("Proton Candidate $\\chi^2_\\mu$");
plt.show();

if dosave:
    plt.savefig(savedir + "%s_otherobj_proton_chi2u.pdf" % DETECTOR);
    plt.savefig(savedir + "%s_otherobj_proton_chi2u.png" % DETECTOR);

In [ ]:
var = df.p.pfp.trk.chi2pid.I2.chi2_proton
bins=np.linspace(0,300,21)

pvar, pw = breakdown_top(var, df.weight, df, cuts)
n, bins, _ = plt.hist(pvar, bins=bins, stacked=True, 
                      label=top_labels, weights=pw)

print("signal purity {:.2f} %".format(100*n[0].sum()/n[-1].sum()))

plt.ylabel("Candidates / %s" % POTSTR);
plt.title("%s Pandora Selection" % DETECTOR);
plt.tight_layout();
plt.legend();
plt.xlabel("Proton Candidate $\\chi^2_p$");
plt.show();

if dosave:
    plt.savefig(savedir + "%s_otherobj_proton_chi2p.pdf" % DETECTOR);
    plt.savefig(savedir + "%s_otherobj_proton_chi2p.png" % DETECTOR);


In [ ]:
# muon cut on muon candidates
MUSEL_MUSCORE_TH = 25
MUSEL_PSCORE_TH = 100
MUSEL_LEN_TH = 50

# TODO: use scores of all 3 planes
# muon_chi2 = (Avg(df, "muon", drop_0=True) < MUSEL_MUSCORE_TH) & (Avg(df, "proton", drop_0=True) > MUSEL_PSCORE_TH)

# TODO: used BDT scores
# len_cut = (masterdf.len.squeeze() > MUSEL_LEN_TH)
# dazzle_muon = (masterdf.dazzle.muonScore > 0.6)
# muon_cut = (muon_chi2) & (len_cut | dazzle_muon)

mu_score_cut = (df.mu.pfp.trk.chi2pid.I2.chi2_muon < MUSEL_MUSCORE_TH) & (df.mu.pfp.trk.chi2pid.I2.chi2_proton > MUSEL_PSCORE_TH)
mu_len_cut = (df.mu.pfp.trk.len > MUSEL_LEN_TH)
mu_cut = (mu_score_cut) & (mu_len_cut)

# proton cut on proton candidates
PSEL_MUSCORE_TH = 0
PSEL_PSCORE_TH = 90

p_score_cut = (df.p.pfp.trk.chi2pid.I2.chi2_muon > PSEL_MUSCORE_TH) & (df.p.pfp.trk.chi2pid.I2.chi2_proton < PSEL_PSCORE_TH) 
p_cut = p_score_cut

# select slices with mu+p
slc_mu_cut = mu_cut.groupby(level=[0,1,2]).any()
slc_p_cut = p_cut.groupby(level=[0,1,2]).any()

cuts += [slc_mu_cut & slc_p_cut]

In [ ]:
var = df.mu.pfp.trk.P.p_muon
bins = np.linspace(0,2,21)

pvar, pw = breakdown_top(var, df.weight, df, cuts)
n, bins, _ = plt.hist(pvar, bins=bins, stacked=True, 
                      label=top_labels, weights=pw)

print("signal purity {:.2f} %".format(100*n[0].sum()/n[-1].sum()))

plt.ylabel("Candidates / %s" % POTSTR);
plt.title("%s Pandora Selection" % DETECTOR);
plt.tight_layout();
plt.legend();
plt.xlabel("Muon Momentum [GeV/c]");
plt.show();

if dosave:
    plt.savefig(savedir + "%s_pid_muon_momentum.pdf" % DETECTOR);
    plt.savefig(savedir + "%s_pid_muon_momentum.png" % DETECTOR);


## Stubs cut

In [ ]:
l_cuts = [5.5e5, 4e5, 3e5, 0]
cut_list = []
for i,l in enumerate(["0_5", "1", "2", "3"]):
    this_cut = np.invert((df.stub["l"+l+"cm"].Q/df.stub["l"+l+"cm"].length) > l_cuts[i])
    cut_list.append(this_cut)
    
cuts += [cut_all(cut_list)]

In [ ]:
var = df.mu.pfp.trk.P.p_muon
bins=np.linspace(0,2,21)

pvar, pw = breakdown_top(var, df.weight, df, cuts)
n, bins, _ = plt.hist(pvar, bins=bins, stacked=True, 
                      label=top_labels, weights=pw)

print("signal purity {:.2f} %".format(100*n[0].sum()/n[-1].sum()))

plt.ylabel("Candidates / %s" % POTSTR);
plt.title("%s Pandora Selection" % DETECTOR);
plt.tight_layout();
plt.legend();
plt.xlabel("Muon Momentum [GeV/c]");
plt.show();

if dosave:
    plt.savefig(savedir + "%s_stub_muon_momentum.pdf" % DETECTOR);
    plt.savefig(savedir + "%s_stub_muon_momentum.png" % DETECTOR);


## Transverse momentum imbalance cut

In [ ]:
DELP_TH = 0.25

In [ ]:
var = df.del_p
bins=np.linspace(0,1,21)

pvar, pw = breakdown_mode(var, df.weight, df, cuts)
n, bins, _ = plt.hist(pvar, bins=bins, stacked=True, 
                      label=mode_labels, color=mode_colors, weights=pw)

print("signal purity {:.2f} %".format(100*n[0].sum()/n[-1].sum()))

plt.ylabel("Candidates / %s" % POTSTR);
plt.title("%s Pandora Selection" % DETECTOR);
plt.tight_layout();
plt.legend();
plt.xlabel("$\\delta p_T$ [GeV/c]");
plt.axvline(DELP_TH, color="k", linestyle="--")
plt.show();

if dosave:
    plt.savefig(savedir + "%s_stub_transverse_momentum_permode.pdf" % DETECTOR);
    plt.savefig(savedir + "%s_stub_transverse_momentum_permode.png" % DETECTOR);


In [ ]:
var = df.del_p
bins=np.linspace(0,1,21)

pvar, pw = breakdown_top(var, df.weight, df, cuts)
n, bins, _ = plt.hist(pvar, bins=bins, stacked=True, 
                      label=top_labels, weights=pw)

print("signal purity {:.2f} %".format(100*n[0].sum()/n[-1].sum()))

plt.ylabel("Candidates / %s" % POTSTR);
plt.title("%s Pandora Selection" % DETECTOR);
plt.tight_layout();
plt.legend();
plt.xlabel("$\\delta p_T$ [GeV/c]");
plt.axvline(DELP_TH, color="k", linestyle="--")
plt.show();

if dosave:
    plt.savefig(savedir + "%s_stub_transverse_momentum.pdf" % DETECTOR);
    plt.savefig(savedir + "%s_stub_transverse_momentum.png" % DETECTOR);


In [ ]:
# transverse momentum cut
delp_cut = (df.del_p < DELP_TH)

cuts += [delp_cut]

In [ ]:
var = df.mu.pfp.trk.P.p_muon
bins=np.linspace(0,2,21)

pvar, pw = breakdown_top(var, df.weight, df, cuts)
n, bins, _ = plt.hist(pvar, bins=bins, stacked=True, 
                      label=top_labels, weights=pw)

print("signal purity {:.2f} %".format(100*n[0].sum()/n[-1].sum()))

plt.ylabel("Candidates / %s" % POTSTR);
plt.title("%s Pandora Selection" % DETECTOR);
plt.tight_layout();
plt.legend();
plt.xlabel("Muon Momentum [GeV/c]");
plt.show();

if dosave:
    plt.savefig(savedir + "%s_dpT_muon_momentum.pdf" % DETECTOR);
    plt.savefig(savedir + "%s_dpT_muon_momentum.png" % DETECTOR);


In [ ]:
import importlib
importlib.reload(kinematics)

In [ ]:
var = kinematics.neutrino_energy(df.mu.pfp.trk.P.p_muon, df.mu.pfp.trk.dir, df.p.pfp.trk.P.p_proton, df.p.pfp.trk.dir)
bins=np.linspace(0,3,21)

pvar, pw = breakdown_top(var, df.weight, df, cuts)
n, bins, _ = plt.hist(pvar, bins=bins, stacked=True, 
                      label=top_labels, weights=pw)

print("signal purity {:.2f} %".format(100*n[0].sum()/n[-1].sum()))

plt.ylabel("Candidates / %s" % POTSTR);
plt.title("%s Pandora Selection" % DETECTOR);
plt.tight_layout();
plt.legend();
plt.xlabel("Neutrino Energy [GeV]");
plt.show();

if dosave:
    plt.savefig(savedir + "%s_dpT_neutrino_energy.pdf" % DETECTOR);
    plt.savefig(savedir + "%s_dpT_neutrino_energy.png" % DETECTOR);


In [ ]:
# Efficiency

In [ ]:
cutnames = [
    "Pre-Selection",
    "Othr Obj. Cut",
    "PID",
    "Stub",
    "$\\delta p_T$"
]

assert(len(cuts) == len(cutnames))

In [ ]:
bins = np.array([0.2, 0.4, 0.6, 0.8, 1, 1.2, 1.5, 2, 2.5, 3])
centers = (bins[:-1] + bins[1:]) /2

varmc = mcdf.E
vardf = df.slc.truth.E

whenmc = is_signal(mcdf)

Ns = []
for i in range(len(cuts)):
    whendf = is_signal(df.slc.truth) & cut_all(cuts[:i+1])
    N,_ = np.histogram(vardf[whendf], weights=df.weight[whendf], bins=bins)
    Ns.append(N)
    
D,_ = np.histogram(varmc[whenmc], weights=mcdf.weight[whenmc], bins=bins)

In [ ]:
for l, N in zip(cutnames, Ns):
    plt.plot(centers, N/D, label=l);
    

plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.225), ncol=3);
plt.xlabel("Neutrino Energy [GeV]");
plt.ylabel("Selection Efficiency");
plt.text(0.025, 0.875, DETECTOR + "\nPandora Selection", transform=plt.gca().transAxes, size=16)
plt.ylim([0, 1])
plt.show();

if dosave:
    plt.savefig(savedir + "%s_efficiency_nuE.pdf" % DETECTOR, bbox_inches="tight");
    plt.savefig(savedir + "%s_efficiency_nuE.png" % DETECTOR, bbox_inches="tight");

In [ ]:
bins = np.array([0.2, 0.4, 0.6, 0.8, 1, 1.2, 1.5, 2, 2.5, 3])
centers = (bins[:-1] + bins[1:]) /2

varmc = mcdf.E
vardf = df.slc.truth.E

whenmc = is_signal(mcdf)

Ns = []
for i in range(1, len(cuts)+1):
    whendf = is_signal(df) & cut_all(cuts[:i] + cuts[i+1:])
    N,_ = np.histogram(vardf[whendf], weights=df.weight[whendf], bins=bins)
    Ns.append(N)
    
D,_ = np.histogram(varmc[whenmc], weights=mcdf.weight[whenmc], bins=bins)

In [ ]:
for l, N in zip(cutnames[1:] + ["All Cuts"], Ns):
    plt.plot(centers, N/D, label=l);
    

plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.225), ncol=3);
plt.xlabel("Neutrino Energy [GeV]");
plt.ylabel("N-1 Efficiency");
plt.text(0.025, 0.875, DETECTOR + "\nML Selection", transform=plt.gca().transAxes, size=16)
plt.ylim([0, 1])
plt.show();

if dosave:
    plt.savefig(savedir + "%s_N-1eff_nuE.pdf" % DETECTOR, bbox_inches="tight");
    plt.savefig(savedir + "%s_N-1eff_nuE.png" % DETECTOR, bbox_inches="tight");